# Model Load & Display

#### Imports

In [1]:
from autotab import TabCNN
import os
import numpy as np
import pandas as pd

2021-11-29 17:44:47.741709: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 17:44:47.741765: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Build The model

In [2]:
my_tabcnn = TabCNN.TabCNN()

Using Google cloud buckets
BUCKET_NAME wagon-data-737-sadriwala
DATA_BUCKET_FOLDER data/spec_repr
gs://wagon-data-737-sadriwala/data/spec_repr/
gs://wagon-data-737-sadriwala/saved/
getting ids from gs://wagon-data-737-sadriwala/data/spec_repr/id.csv


In [3]:
model = my_tabcnn.build_model()

2021-11-29 17:44:53.796303: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-29 17:44:53.796398: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-29 17:44:53.796429: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-GP5BU3LN): /proc/driver/nvidia/version does not exist
2021-11-29 17:44:53.796825: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 190, 7, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 188, 5, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 186, 3, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 93, 1, 64)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 93, 1, 64)         0         
                                                                 
 flatten (Flatten)           (None, 5952)              0         
                                                        

### Load the weights

In [5]:
os.chdir('../h5-model')
!ls

full_val0_75acc_weights.h5


In [6]:
model.load_weights('full_val0_75acc_weights.h5')

### Load the npz File

find the smallest file in 0 folder

In [7]:
import numpy as np
data = np.load(  # load the pre -predicted file
    '/mnt/d/data-science/le-wagon/autotab/data/spec_repr/c/00_Jazz1-200-B_solo.npz',
    allow_pickle=True)
file_list = data.files  # files in data
print(file_list)

['repr', 'labels']


### Extract the repr

In [8]:
repr = data['repr']

In [9]:
repr.shape

(621, 192)

### get nearby frames

In [10]:
con_win_size= my_tabcnn.con_win_size
halfwin = con_win_size // 2
full_x = np.pad(
    repr,
    [
        (halfwin, halfwin
         ),  # full x is the entire song padded with halfwin*2 frames
        (0, 0)
    ],
    mode='constant')


In [11]:
x_new = []
for frame_idx in range(0, len(repr)):  # for all frames in the experiment
    sample_x = full_x[frame_idx:frame_idx + my_tabcnn.con_win_size]
    x_new.append(np.expand_dims(np.swapaxes(sample_x, 0, 1), -1))

In [12]:
print(f"shape = ({len(x_new)},{x_new[0].shape}")

shape = (621,(192, 9, 1)


In [13]:
x_new = np.array(x_new, dtype='float32')


### predict using model.predict on the repr

In [14]:
y_pred = model.predict(x_new)

In [15]:
len(y_pred)

621

### make tabs with the prediction

In [16]:
from autotab.TabPrediction import make_squeezed_tab
from autotab.TabPrediction import make_full_tab
#test it out
expanded_tab = make_full_tab(y_pred, len(y_pred))
expanded_tab

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  tablature[frame_idx] = fret


,0,1,2,3,4,5,6,7,8,9,...,611,612,613,614,615,616,617,618,619,620
E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
G,0,9,9,9,9,9,9,9,9,9,...,0,0,0,0,0,0,0,0,0,0
B,0,5,5,5,5,5,5,5,5,5,...,7,7,0,0,0,0,0,0,0,0
e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
display_tab = make_squeezed_tab(expanded_tab, 3)
display_tab

/mnt/d/data-science/le-wagon/autotab/autotab/TabPrediction.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  squeezed_Tab[batch_idx] = frame_batch.mode(axis='columns')[0]


,0,3,6,9,12,15,18,21,24,27,...,591,594,597,600,603,606,609,612,615,618
E,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,7,0,7,0,9,...,0,0,0,0,0,0,0,0,0,0
G,9,9,9,9,9,9,9,9,9,9,...,7,7,7,7,0,0,0,0,0,0
B,5,5,5,5,5,5,5,5,5,5,...,7,7,7,7,7,7,7,0,0,0
e,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
